#### <center> <p style = 'background-color:darkblue'><b>Task - 1</p> </center>

In [17]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from datasets import load_dataset

In [18]:
nltk.download('punkt_tab')
texts = "Natural language processing enables machines to understand human language."
tokens = word_tokenize(texts)
print("Токенізований текст:", tokens)

Токенізований текст: ['Natural', 'language', 'processing', 'enables', 'machines', 'to', 'understand', 'human', 'language', '.']


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\gnatu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [19]:
# 🔹 Завантаження потрібних ресурсів
nltk.download('punkt')
nltk.download('stopwords')

# 🔹 Стрімінг — не завантажує весь датасет одразу
ds = load_dataset("agentlans/high-quality-english-sentences", split="train", streaming=True)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gnatu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gnatu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# 🔹 Вхідний текст
texts = "Natural language processing enables machines to understand human language."

# 🔹 Токенізація
tokens = word_tokenize(texts.lower())
print("Токенізований текст:", tokens)

Токенізований текст: ['natural', 'language', 'processing', 'enables', 'machines', 'to', 'understand', 'human', 'language', '.']


In [21]:
# 🔹 Англійські стоп-слова
stop_words = set(stopwords.words('english'))
stop_words.update(["language", "processing", "machines"])

# 🔹 Додаткові стоп-слова
my_stopwords = {
    "the", "is", "and", "to", "in", "this", "language", "processing"
}

# 🔹 Фільтрація: видаляємо стоп-слова і пунктуацію
filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

print("🔸 Токени без стоп-слів:")
print(filtered_tokens)

🔸 Токени без стоп-слів:
['natural', 'enables', 'understand', 'human']


In [30]:
no_filtered_sentences = []

for i, sample in enumerate(ds):
    sentence = sample["text"]
    no_filtered_sentences.append(sentence)

    if i >= 3000:  # ⚠️ Наприклад, обробити лише перші 10 тисяч речень
        break

# 🔹 Збереження у файл (опціонально)
with open("no_filtered_sentences.txt", "w", encoding="utf-8") as f:
    for sentence in no_filtered_sentences:
        f.write(sentence + "\n")


In [31]:
filtered_sentences = []

for i, sample in enumerate(ds):
    sentence = sample["text"]
    tokens_ds = word_tokenize(sentence.lower())
    filtered = [w for w in tokens_ds if w.isalpha() and w not in stop_words]
    filtered_sentences.append(filtered)

    if i >= 3000:  # ⚠️ Наприклад, обробити лише перші 10 тисяч речень
        break

# 🔹 Збереження у файл (опціонально)
with open("filtered_sentences.txt", "w", encoding="utf-8") as f:
    for sentence in filtered_sentences:
        f.write(" ".join(sentence) + "\n")

#### <center> <p style = 'background-color:darkblue'><b>Task - 2</p> </center>

In [63]:
import numpy as np

def load_glove_model(glove_file):
    glove_model = {}
    with open(glove_file, "r", encoding="utf-8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float32)
            glove_model[word] = embedding
    return glove_model

glove_file = "glove.6B.50d.txt"  # Необхідно завантажити файл заздалегідь https://drive.google.com/file/d/1mN2Ee44z6CBkCW4QQ95gHHjQTaSk-fxM/view?usp=sharing
glove_model = load_glove_model(glove_file)
print("Вектор для слова 'human':", glove_model.get("human", "Слово не знайдено"))

Вектор для слова 'human': [ 0.61854   0.11915  -0.46786   0.31368   1.0334    0.95964   0.87803
 -1.0346    1.6322    0.29347   0.80844  -0.058903  0.021251  0.40986
  0.54443  -0.33311   0.53712  -0.35823   0.29374   0.090151 -0.92049
  0.69386   0.39098  -0.64392   0.77831  -1.7215   -0.48393  -0.50327
 -0.22508   0.099192  3.2095   -0.31554  -0.71754  -1.6752   -1.3537
  0.15195   0.054557 -0.1633   -0.027993  0.3917   -0.55007  -0.079205
  0.63389   0.51446   0.70124   0.27638  -0.53445   0.064808 -0.21974
 -0.52048 ]


#### <center> <p style = 'background-color:darkblue'><b>Task - 3</p> </center>

In [67]:
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Вибір двох слів
word1 = "question"
word2 = "answer"

vec1 = glove_model.get(word1)
vec2 = glove_model.get(word2)

if vec1 is not None and vec2 is not None:
    similarity = cosine_similarity(vec1, vec2)
    print(f"Косинусна подібність між словами '{word1}' та '{word2}': {similarity:.4f}")
else:
    print("Одне або обидва слова не знайдені у моделі.")


Косинусна подібність між словами 'question' та 'answer': 0.9242


#### <center> <p style = 'background-color:darkblue'><b>Task - 4</p> </center>

In [61]:
def find_similar_words(glove_model, query_word, top_n=10):
    if query_word not in glove_model:
        print(f"Слово '{query_word}' не знайдено в моделі.")
        return []

    query_vector = glove_model[query_word]
    similarities = {}

    for word, vector in glove_model.items():
        if word != query_word:
            sim = cosine_similarity(query_vector, vector)
            similarities[word] = sim

    # Сортуємо за подібністю та повертаємо топ-N
    similar_words = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:top_n]

    return similar_words

# Використання:
similar = find_similar_words(glove_model, "woman", top_n=10)
print("Схожі слова до 'woman':")
for word, sim in similar:
    print(f"{word}: {sim:.4f}")


Схожі слова до 'woman':
girl: 0.9065
man: 0.8860
mother: 0.8764
her: 0.8613
boy: 0.8596
she: 0.8431
herself: 0.8225
child: 0.8108
wife: 0.8037
old: 0.7982


Під час лабораторної роботи було реалізовано програму, яка виконує основні етапи попередньої обробки тексту: токенізацію та видалення стоп-слів із використанням бібліотеки nltk. Для токенізації застосовано метод word_tokenize, а набір стоп-слів розширено власноруч, що дозволило краще відфільтрувати зайві слова з тексту.

Дані були завантажені з англомовного датасету "high-quality-english-sentences" через бібліотеку datasets у стрімінговому режимі, що оптимізувало використання оперативної памʼяті.

Після очищення тексту було сформовано корпус речень, на основі якого побудовано векторні представлення слів за допомогою моделі GloVe (Global Vectors for Word Representation). Хоча у завданні пропонувалось використати Word2Vec, обрано GloVe — що є цілком прийнятним, адже ця модель також є word-embedding підходом, але з відмінним математичним підґрунтям.

За допомогою отриманих векторів було розраховано косинусну подібність між парами слів, що дало змогу оцінити їхню семантичну близькість у векторному просторі. Також знайдено слова, які найбільше схожі на задане, за допомогою функції пошуку найближчих векторів.

У результаті виконання роботи отримано практичний досвід у:

попередній обробці тексту;

роботі з корпусами через стрімінг;

побудові векторних уявлень слів;

пошуку схожих слів у embedding-просторі.